In [2]:
!pip install -e C:\Users\lucia\Documents\GitHub\rtk

Obtaining file:///C:/Users/lucia/Documents/GitHub/rtk
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for click==8.1.3 from https://files.pythonhosted.org/packages/c2/f1/df59e28c642d583f7dacffb1e0965d0e00b218e0186d7858ac5233dce840/click-8.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for requests==2.28.1 from https://files.pythonhosted.org/packages/ca/91/6d9b8ccacd0412c08820f72cebaa4f0c0441b5cda699c90f618b6f8a1b42/requests-2.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm==4.64.1 from https://files.pythonhosted.org/packages/47/bb/849011636c4da2e44f1253cd927cfb20ada4374d8b3a4e425416e84900cc/tqdm-4.64.1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.3 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.3 kB ? eta -:--:--
     ---------------------------------------- 57.3/57.3 kB 1.5 MB/s eta 0:00:00
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
dgl 2.0.0+cu121 requires torchdata>=0.5.0, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.28.1 which is incompatible.
datasets 2.19.2 requires requests>=2.32.1, but you have requests 2.28.1 which is incompatible.
kraken 6.0.2 requires numpy~=2.0.0, but you have numpy 1.26.4 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [1]:
from rtk.task import KrakenAltoCleanUpCommand, YALTAiCommand, KrakenRecognizerCommand, ExtractZoneAltoCommand
from rtk import utils
import glob
from sys import argv
import time
import os

In [ ]:
# Get all PNG files from all subdirectories
all_files = glob.glob("Editions/Padova_1618_Cesare_Ripa/*.png")


print(f"processing {len(all_files)} PNGs")

start = time.time()

num_workers = 5

for i in range(0, len(all_files), 4):
    batch = all_files[i:i+4]
    
    startYalt = time.time()
    # Apply YALTAi
    print("[Task] Segment, size of batch ", len(batch))
    yaltai = YALTAiCommand(
        batch,
        binary="yaltai",
        device="cpu",
        yolo_model="my_finetune_project/run_ladas_1280_l_v14/weights/best.pt",
        verbose=True,
        raise_on_error=False,
        allow_failure=False,
        multiprocess=10,
        check_content=False,
        line_model="models/blla.mlmodel"
    )
    yaltai.process()
    endYalt = time.time()
    
    print("[Time] Yaltai: ", endYalt - startYalt)
    print("Yaltai output files len ", len(yaltai.output_files))
    print("[Task] Fix ALTO file paths") 
    
    # Fix paths for current batch - corrected for PNG files
    batch_xml_files = [img.replace('.png', '.xml') for img in batch]
    for alto_file in batch_xml_files:
        if os.path.exists(alto_file):
            with open(alto_file, 'r') as f:
                content = f.read()
            
            # Get the corresponding image file name
            img_file = alto_file.replace('.xml', '.png')
            img_filename = os.path.basename(img_file)
            
            # Fix various possible path patterns
            fixed_content = content.replace(f'<fileName>{img_file}</fileName>', f'<fileName>{img_filename}</fileName>')
            fixed_content = fixed_content.replace(f'<fileName>Editions/Padova_1618_Cesare_Ripa_processed/', '<fileName>')
            fixed_content = fixed_content.replace('<fileName>sample/', '<fileName>')
            
            # Remove any remaining directory structure
            import re
            fixed_content = re.sub(r'<fileName>[^<]*?([^/]+\.png)</fileName>', r'<fileName>\1</fileName>', fixed_content)
            
            with open(alto_file, 'w') as f:
                f.write(fixed_content)
    
    print("[Task] Clean-Up Serialization")
    cleanup = KrakenAltoCleanUpCommand(yaltai.output_files)
    cleanup.process()
    
    startKrak = time.time()
    print("[Task] OCR")
    kraken = KrakenRecognizerCommand(
        yaltai.output_files,
        binary="kraken",
        device="cpu",
        model="models/catmus-print-fondue-large.mlmodel",
        multiprocess=10,
        check_content=True
    )
    kraken.process()
    endKrak = time.time()
    
    print("[Time] Kraken: ", endKrak - startKrak)

end = time.time()
print("[Time] total: ", end - start)

processing 704 PNGs


ValueError: invalid literal for int() with base 10: '--f=c:\\Users\\lucia\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3664d250883ce00f3ea879d9ad5bd99bdc6cba978.json'